# Import Packages and Data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import chi2_contingency
from itertools import combinations
import math

from IPython.display import HTML, display

# read Loyalty# as string because it's an identifier and we never perform numerical operations on it
df_customer = pd.read_csv("DM_AIAI_CustomerDB.csv", dtype={"Loyalty#": str})
df_flights = pd.read_csv("DM_AIAI_FlightsDB.csv", dtype={"Loyalty#": str})

# display(df_customer.info())
display(df_flights.info())
df_flights.head()

In [0]:
# Rename columns due to programming standards while keeping the meaning
# df_customer.info()
df_customer = df_customer.rename(columns={
    'Loyalty#': 'loyalty_id',
    'First Name': 'first_name',
    'Last Name': 'last_name',
    'Customer Name': 'customer_name',
    'Country': 'country',
    'Province or State': 'prov_or_state',
    'City': 'city',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'Postal code': 'postal_code',
    'Location Code': 'location_code',
    'Gender': 'gender',
    'Education': 'education',
    'Income': 'income',
    'Marital Status': 'marital_status',
    'LoyaltyStatus': 'loyalty_status',
    'EnrollmentDateOpening': 'enrollment_date',
    'CancellationDate': 'cancellation_date',
    'Customer Lifetime Value': 'lifetime_value',
    'EnrollmentType': 'enrollment_type'
})
df_customer.info()

In [0]:
df_flights = df_flights.rename(columns={
    'Loyalty#': 'loyalty_id',
    'Year': 'year',
    'Month': 'month',
    'YearMonthDate': 'year_month_date',
    'NumFlights': 'num_flights',
    'NumFlightsWithCompanions': 'num_flights_with_companions',
    'DistanceKM': 'distance_km',
    'PointsAccumulated': 'points_accumulated',
    'PointsRedeemed': 'points_redeemed',
    'DollarCostPointsRedeemed': 'dollar_cost_points_redeemed'
})
df_flights.info()

# First basic Data Exploration of the two raw dataframes

In [0]:
# TODO Prio low: maybe outsource in function and call it with both dataframes to avoid redundant code

## Data Exploration and basic cleaning of df_customer

In [0]:
# replace potential empty strings with nan first (Missing values will be handled later when preparing the data for the model)
df_customer = df_customer.replace(r'^\s*$', np.nan, regex=True) # use regex to catch empty or whitespace-only strings
df_customer.info()
# => 20 null values in lifetime_value and Income, 14.327 null values in cancellation_date (still active loyalty members)

# columns enrollment_date and cancellation_date need to be converted to Datetime (coerce sets invalid parsing to NaT)
df_customer["enrollment_date"] = pd.to_datetime(df_customer["enrollment_date"], errors="coerce")
df_customer["cancellation_date"] = pd.to_datetime(df_customer["cancellation_date"], errors="coerce")

In [0]:
df_customer.describe(include="all").T

# Findings:
#   loyalty_id: 100.011 to 999.999: weird because only 16.921 IDs
#   first_name/last_name/customer_name: CN completely unique, 1.517 people with shared last names (families?)
#   location data: Almost 1/3 from Ontario (Toronto), all from Canada
#   latitude/longitude: std of Longitude is 22, Customers are widely spread east to west > large coverage of Canada
#   gender/education/marital_status: Male/Female equally distributed, most are Bachelors and Married 
#   income: High difference of median/mean, high std, 25% = 0 > do they really have no income or is it missing/censored data? max income 99.981 > no outliers to right side, capped?
#   enrollment_date/cancellation_date: Enrollment Range from 2015–2021 (7 years), 86.4% of all Customers still active 
#   lifetime_value: Mean 7.990, Std 5.780 > Some Customers are worth much more
#   enrollment_type: Standard Dominates


In [0]:
### Remove duplicates and constant variables
print(len(df_customer))
print(f"\nDuplicate customer rows: {df_customer.duplicated().sum()}\n")
print(f"Duplicate customer on loyalty_id: {df_customer['loyalty_id'].duplicated().sum()}\n")

# Before further data exploration, removal of duplicate customer loyalty_id (327 of 16.921) needed because no mapping possible
duplicate_loyalty_ids = df_customer['loyalty_id'][df_customer['loyalty_id'].duplicated()] # Store them in variable to also remove them from df_flights
df_customer = df_customer[~df_customer['loyalty_id'].isin(duplicate_loyalty_ids)]
print(len(df_customer))

# Remove column country here because canada is the only value and therefore the column is constant and useless for modeling
df_customer = df_customer.drop(columns=['country'])
df_customer = df_customer.drop(columns=['Unnamed: 0'])

## Data Exploration and basic cleaning of df_flights

In [0]:
# replace potential empty strings with nan first (Missing values will be handled later when preparing the data for the model)
df_flights = df_flights.replace(r'^\s*$', np.nan, regex=True) # use regex to catch empty or whitespace-only strings
df_flights.info()
# => no null values

# column year_month_date needs to be converted to Datetime (keep Year and Month as integers because its more memory efficient)
df_flights["year_month_date"] = pd.to_datetime(df_flights["year_month_date"], errors="coerce")

In [0]:
df_flights.describe(include="all").T

# Findings:
#   year_month_date: Range from 01-01-2019 to 01-12-2021 
#   distance_km: max 42.000km in a month seems to be an outlier
#   points_accumulated: Most customers dont accumulate lots of points, top flyers dominate, mean way higher than avg
#   Redeemed: points_redeemed = 0, lots of customers never redeem points

In [0]:
### Remove duplicates
print(len(df_flights))
# Drop removed loyalty_ids from df_customer from df_flights as well
df_flights = df_flights[~df_flights['loyalty_id'].isin(duplicate_loyalty_ids)]
print(len(df_flights))
# => from 608436 to 596664

print(f"\nDuplicate flight rows: {df_flights.duplicated().sum()}\n")
# => No duplicate flight rows anymore

## Check for overlaps or inconsistencies between the two dataframes

In [0]:
# Find matches between both dataframes based on loyalty_id
cust_ids = set(df_customer["loyalty_id"])
flight_ids = set(df_flights["loyalty_id"])

# Matches
common_ids = cust_ids & flight_ids
only_in_customers = cust_ids - flight_ids
only_in_flights = flight_ids - cust_ids

print(f"Total in customers: {len(cust_ids)}")
print(f"Total in flights:   {len(flight_ids)}")
print(f"Matching IDs:       {len(common_ids)}")
print(f"Only in customers:  {len(only_in_customers)}")
print(f"Only in flights:    {len(only_in_flights)}\n")
# => 20 customers without flights (16594 in customer data but only 16574 in flight data)

# Customers present in customer data but missing in flights
missing_flight_customers = df_customer[df_customer['loyalty_id'].isin(only_in_customers)]
print(f"Customers in customers but not in flights: {len(missing_flight_customers)}")
display(missing_flight_customers)
# => These 20 customers enrolled and cancelled on the exact same day

### Remove sparse months of flights per customer before their (first flight | enrollment_date) to delete useless data # TODO check whether this changes something in EDA and is necessary for now or later when preparing the data for modelling
# => Suggestion for Business: Adjust data management by not adding past empty data for new customers

# Join data (aggregate flight data for each customer)

In [0]:
# Merge both dataframes to get a df containing all flights mapped to customer info (~600.000 entries: each customer has 36 rows, one per month)
df_customerflights = df_flights.merge(df_customer, on="loyalty_id", how="inner") # inner join excludes the 20 customers without flights because they are not important analysis regarding flights
print(len(df_customerflights))
df_customerflights.head()

In [0]:
# Aggregate (sum) the numerical flight data per customer (~16.000 entries) 
# year, month and year_month_date are dropped: Not relevant for aggregation but for time series analysis later
df_aggregated_flights_per_customer = df_flights.groupby("loyalty_id").agg({
    'num_flights': 'sum',
    'num_flights_with_companions': 'sum',
    'distance_km': 'sum',
    'points_accumulated': 'sum',
    'points_redeemed': 'sum',
    'dollar_cost_points_redeemed': 'sum',
}).reset_index().rename(columns={
    'num_flights': 'total_num_flights',
    'num_flights_with_companions': 'total_num_flights_with_companions',
    'distance_km': 'total_distance_km',
    'points_accumulated': 'total_points_accumulated',
    'points_redeemed': 'total_points_redeemed',
    'dollar_cost_points_redeemed': 'total_dollar_cost_points_redeemed'
}) # reset_index to have loyalty_id as a column and rename the columns for clarity
print(f'Customer dataframe ({len(df_customer)} entries):')
display(df_customer.head())
print(f'Aggregated (summed) flights grouped per customer dataframe ({len(df_aggregated_flights_per_customer)} entries):')
display(df_aggregated_flights_per_customer.head())
df_cf_aggregated = df_customer.merge(df_aggregated_flights_per_customer, on="loyalty_id", how="left")
print(f'Merged customer and aggregated flights dataframe ({len(df_cf_aggregated)} entries):')
display(df_cf_aggregated.head())
display(df_cf_aggregated.info())

In [0]:
# Quick summary statistics of the merged dataframe before feature engineering and EDA of the final dataframe
df_cf_aggregated.describe(include="all").T

# Data exploration of the joined dataframes

In [0]:
# how many customers have a cancellation_date?
ex_loyalty_customers = df_cf_aggregated.loc[df_cf_aggregated["cancellation_date"].notna(), "loyalty_id"]
print("Number of Ex-Loyalty Customers:", len(ex_loyalty_customers), "\n")
# => 2265 (while 20 of those have 0 flights and were enrolled for only one day -> maybe a mistake or some incentives for enrollment but then cancelled immediately)


# how many customers have no flights at all?
customers_no_flights = df_cf_aggregated[df_cf_aggregated['total_num_flights'] == 0]
print(f"Number of customers with zero flights: {len(customers_no_flights)}\n")
# => 1473 customers with zero flights (probably they just enrolled but never used the loyalty program or cancelled immediately or are new customers)


# how many customers have no cancellation_date but also no flights? -> enrolled but inactive (Cold Start Program)
inactive_but_enrolled_customers_ = df_cf_aggregated[
    (df_cf_aggregated["cancellation_date"].notna()) &
    (df_cf_aggregated["total_num_flights"] == 0)
]
print("Number of inactive but enrolled Customers for Cold Start Program:", len(inactive_but_enrolled_customers_), "\n")
# => 945 inactive but enrolled Customers for Cold Start Program


# how many customers have a cancellation_date but have flights afterwards? > un-enrolled but still active (Welcome Back Program) (use df_customerflights to check monthly data)
active_ex_customers = df_customerflights.groupby("loyalty_id").filter(
    lambda customer: customer["cancellation_date"].notna().any() and 
              ((customer["num_flights"] > 0) & (customer["year_month_date"] > customer["cancellation_date"])).any()
)
print("Number of active ex-Customers for Welcome Back Program:", active_ex_customers["loyalty_id"].nunique(), "\n")
# => 174 un-enrolled but still active (Welcome Back Program)


# Customers with Flights before their enrollment_date (use df_customerflights to check monthly data)
customers_with_flights_before_enrollment = df_customerflights[
    (df_customerflights["year_month_date"] < df_customerflights["enrollment_date"]) &
    (df_customerflights["num_flights"] > 0)
]["loyalty_id"].unique()
print("Number of Customers with Flights before Enrollment:", len(customers_with_flights_before_enrollment), "\n")
# => 4981 Customers with flights before enrollment


# Customers with cancellation_date before their enrollment_date
customers_with_cancellation_before_enrollment = df_cf_aggregated[df_cf_aggregated["cancellation_date"] < df_cf_aggregated["enrollment_date"]]
print("Number of Customers with Cancellation before Enrollment:", len(customers_with_cancellation_before_enrollment), "\n")
# => 199 customers with cancellation before enrollment (does not make sense and should be checked with business -> probably wrong data entry)

# Feature engineering

## WIP ~Jan: On df_cf_aggregated (to be removed or finished entirely (Prio low))

In [0]:
# Adding necessary aggregations of columns from the flights dataset as new features (total, avg, median, std)
# Build a lot of features first and remove them later through feature selection as discussed in the lecture

# Flights
median_flights_per_month = df_flights.groupby('loyalty_id')['num_flights'].median()
std_flights_per_month = df_flights.groupby('loyalty_id')['num_flights'].std()
flights_aggs = pd.DataFrame({
    'median_flights_per_month': median_flights_per_month,
    'std_flights_per_month': std_flights_per_month
})
df_cf_aggregated = df_cf_aggregated.merge(flights_aggs, on='loyalty_id', how='inner')

# # Uncomment this display-check to check whether both calculations for df_cf_aggregated and df_customerflights are the same
# display(df_cf_aggregated[['loyalty_id', 'avg_flights_per_month', 'median_flights_per_month', 'std_flights_per_month']].sort_values('loyalty_id').head())
# display(df_customerflights[['loyalty_id', 'avgflightsmonth', 'median_flights_per_month', 'std_flights_per_month']].drop_duplicates(subset=['loyalty_id']).sort_values('loyalty_id').head())

# TODO Prio low: to be continued by Jan from here to have all engineered features in the aggregated df immediately instead of the df_customerflights

display(df_cf_aggregated.columns)

## On df_customerflights

In [0]:
# Adding necessary aggregations of columns from the flights dataset as new features (total, median, std)
# Build a lot of features first and remove them later through feature selection as discussed in the lecture

# Flights
# Mean is not calculated because its the total value divided by all months (36) so their are perfectly correlated leading to zero new information
df_customerflights['total_flights'] = df_customerflights.groupby('loyalty_id')['num_flights'].transform('sum')
df_customerflights['median_flights_per_month'] = df_customerflights.groupby('loyalty_id')['num_flights'].transform('median')
df_customerflights['std_flights_per_month'] = df_customerflights.groupby('loyalty_id')['num_flights'].transform('std')

# # Uncomment to check that mean and total/36 is the same information
# df_customerflights['avgflightsmonth'] = df_customerflights.groupby('loyalty_id')['num_flights'].transform('mean').round(2)
# df_customerflights['total_flights_div_36'] = (df_customerflights['total_flights'] / 36).round(2)
# df_customerflights[df_customerflights['avgflightsmonth'] != df_customerflights['total_flights_div_36']]
# display(df_customerflights[['loyalty_id', "avgflightsmonth", 'total_flights_div_36']][df_customerflights['avgflightsmonth'] == df_customerflights['total_flights_div_36']])

# Flights with Companions (Number of flights with companions within a month)
df_customerflights['total_flights_w_comp'] = df_customerflights.groupby('loyalty_id')['num_flights_with_companions'].transform('sum')
df_customerflights['median_flights_w_comp'] = df_customerflights.groupby('loyalty_id')['num_flights_with_companions'].transform('median')
df_customerflights['std_flights_w_comp'] = df_customerflights.groupby('loyalty_id')['num_flights_with_companions'].transform('std')

# Distance
df_customerflights['total_distance_km'] = df_customerflights.groupby('loyalty_id')['distance_km'].transform('sum')
df_customerflights['median_distance_km'] = df_customerflights.groupby('loyalty_id')['distance_km'].transform('median')
df_customerflights['std_distance_km'] = df_customerflights.groupby('loyalty_id')['distance_km'].transform('std')

# Points Accumulated
df_customerflights['total_points_acc'] = df_customerflights.groupby('loyalty_id')['points_accumulated'].transform('sum')
df_customerflights['median_points_acc'] = df_customerflights.groupby('loyalty_id')['points_accumulated'].transform('median')
df_customerflights['std_points_acc'] = df_customerflights.groupby('loyalty_id')['points_accumulated'].transform('std')

# Points Redeemed
df_customerflights['total_points_redeemed'] = df_customerflights.groupby('loyalty_id')['points_redeemed'].transform('sum')
df_customerflights['median_points_redeemed'] = df_customerflights.groupby('loyalty_id')['points_redeemed'].transform('median')
df_customerflights['std_points_redeemed'] = df_customerflights.groupby('loyalty_id')['points_redeemed'].transform('std')

# Dollar Cost Points Redeemed
df_customerflights['total_dollar_redeemed'] = df_customerflights.groupby('loyalty_id')['dollar_cost_points_redeemed'].transform('sum')
df_customerflights['median_dollar_redeemed'] = df_customerflights.groupby('loyalty_id')['dollar_cost_points_redeemed'].transform('median')
df_customerflights['std_dollar_redeemed'] = df_customerflights.groupby('loyalty_id')['dollar_cost_points_redeemed'].transform('std')

display(df_customerflights)

In [0]:
# TODO maybe squared features

In [0]:
### Engineered features using the total_flights:
# add a column ratio_comp_per_flight that shows the ratio of flights with companions to total flights
df_customerflights['ratio_comp_per_flight'] = np.where(
    df_customerflights['total_flights'] > 0,
    (df_customerflights['total_flights_w_comp'] / df_customerflights['total_flights']).round(4),
    np.nan
)
df_customerflights['ratio_comp_per_flight'].describe()
# => This feature could provide insights about customers that often fly with companions (e.g. families, couples)

# add a column that provides insights on the average distance flown per flight
df_customerflights['avg_distance_per_flight'] = np.where(
    df_customerflights['total_flights'] > 0,
    (df_customerflights['total_distance_km'] / df_customerflights['total_flights']).round(2),
    np.nan
)
# => This feature could provide insights about customers that often fly long vs. short distances

# add a column that provides insights on the average total_points_acc selected per flight
df_customerflights['avg_points_acc_per_flight'] = np.where(
    df_customerflights['total_flights'] > 0,
    (df_customerflights['total_points_acc'] / df_customerflights['total_flights']).round(2),
    np.nan
)
# => This feature could provide insights about customers that often accumulate many points per flight # TODO check correlation with avg_distance_per_flight

# add a column that shows the clv per flight (only makes sense if total_flights is not part of the formula for clv)
df_customerflights['lifetime_value_per_flight'] = np.where(
    df_customerflights['total_flights'] > 0,
    (df_customerflights['lifetime_value'] / df_customerflights['total_flights']).round(2),
    np.nan
)
df_customerflights['lifetime_value_per_flight'].describe()
df_customerflights['lifetime_value'].describe()
# => This feature could provide insights about how valuable a customer is per flight taken

In [0]:
# add column is_enrolled (bool, enrollment_date but no cancellation_date)
df_customerflights['is_enrolled'] = df_customerflights['cancellation_date'].isna() & df_customerflights['enrollment_date'].notna()

# add column recency (int, in months, time since last num_flights)
df_customerflights = df_customerflights.sort_values(['loyalty_id', 'year_month_date'], ascending=[True, False])
target_date = pd.Timestamp('2021-12-01')
df_customerflights['recency'] = pd.NA

# add column recency, group by loyaltyid, yearmonthdate desc, count rows until num_flights != 0
for loyalty_id, group in df_customerflights.groupby('loyalty_id'):
    if target_date in group['year_month_date'].values:
        rec = 0
        for f in group['num_flights'].values:
            if f > 0:
                break
            rec += 1
        idx = group[group['year_month_date'] == target_date].index
        df_customerflights.loc[idx, 'recency'] = rec

df_customerflights['recency'] = df_customerflights['recency'].astype('Int64') # Store as int instead of object

# add column active_months (int, in months, count number of rows where num_flights >0)
df_customerflights['active_months'] = df_customerflights.groupby('loyalty_id')['num_flights']\
    .transform(lambda x: (x > 0).sum())

# add column enrolled_since_days (int, in days, time since enrollment_date until 2021-12-30 (max value of enrollment_date while max of df_flights is 2021-12-01))
df_customerflights['enrolled_since_days'] = ((pd.Timestamp('2021-12-30') - df_customerflights['enrollment_date']) / np.timedelta64(1, 'D')).astype('Int64')
print(df_customerflights['enrollment_date'].max())
# => This feature could provide insights about clv and total_flights

# add column cancelled_since_days (int, in days, time since cancellation_date until 2021-12-30 (max value of cancellation_date while max of df_flights is 2021-12-01))
df_customerflights['cancelled_since_days'] = np.where(
    df_customerflights['cancellation_date'].notna(),
    ((pd.Timestamp('2021-12-30') - df_customerflights['cancellation_date']) / np.timedelta64(1, 'D')).astype('Int64'),
    pd.NA
)
print(df_customerflights['cancellation_date'].max())
# => This feature could provide insights about retention time and loyalty programs

# add a column that shows the ratio of points redeemed to points accumulated
df_customerflights['ratio_points_redeemed'] = np.where(
    df_customerflights['total_points_acc'] > 0,
    (df_customerflights['total_points_redeemed'] / df_customerflights['total_points_acc']).round(4),
    np.nan
)
df_customerflights['ratio_points_redeemed'].describe() # TODO the max value of 33 is not possible as it is > 1
# => This feature could provide insights about customer engagement in the loyalty program (=> activate people that do not redeem points to get the to collect more too and therefore fly more often with AIAI)


# TODO add a feature clv per distance => could provide insights into why the clv does not make sense (business: several short flights could sum up to one long-distance flights in km but not in value for the company) => challenge the company formula for clv and find out state-of-the-art clv formula -> clv should be aligned with money and not with km

# add a column that shows the frequency as a ratio of a customer since enrollment ("active months since enrollment") # TODO


# Data Exploration of final dataframe including the engineered features

In [0]:
### Creation of df_cf_unique (final dataframe)
# Due to the reason that df_customer only contains current data, a mapping of the customer to all their historic flight data is not useful. a combination of customer data together with total and average flight data is needed.
df_cf_unique = df_customerflights.drop_duplicates(subset=['loyalty_id'], keep='first').drop(
    columns=['year', 'month', 'year_month_date', 'num_flights', 'num_flights_with_companions',
             'distance_km', 'points_accumulated', 'points_redeemed', 'dollar_cost_points_redeemed']
)
# => The 20 customers who didn't fly at all and were only present in the customers.csv but not in the flights.csv are not included in the final dataframe for analysis

display(df_cf_unique.head())
df_cf_unique.info()
# => Final dataframe for further analysis with 16.574 unique customers with aggregated flight data

df_cf_unique.describe(include="all").T

In [0]:
# TODO add this EDA under the creation of the df_cf_unique to do this before the feature engineering
# Customers in df_cf_unique with enrollment_date before 2021 but enrollment_type "2021 Promotion"
customers_enroll_before_2021_but_promotion = df_cf_unique[
    (df_cf_unique["enrollment_date"] < "2021-01-01") &
    (df_cf_unique["enrollment_type"] == "2021 Promotion")
]["loyalty_id"]
print("Number of Customers enrolled before 2021 but with '2021 Promotion':", len(customers_enroll_before_2021_but_promotion), "\n")

# Customers where points_redeemed > points_accumulated
customers_points_redeemed_greater_than_totalpoints = df_cf_unique[df_cf_unique["total_points_redeemed"] > df_cf_unique["total_points_acc"]
]["loyalty_id"]
print("Number of Customers with pointsredeemed > pointsaccumulated:", len(customers_points_redeemed_greater_than_totalpoints), "\n") # TODO verbindung zu dem max=33 wert bei der ratio herstellen

# Customers where total_flights_w_comp > num_flights
customers_totalflightscomp_greater_than_totalflights = df_cf_unique[
    df_cf_unique["total_flights_w_comp"] > df_cf_unique["total_flights"]
]["loyalty_id"]
print("Number of Customers with total_flights_w_comp > total_flights:", len(customers_totalflightscomp_greater_than_totalflights), "\n")

# Extreme Distances in flights (customers with very little and very long distances per flight) # TODO maybe this is redundant with later outlier detection
extreme_avg_flight_distance = df_cf_unique[(df_cf_unique['avg_distance_per_flight'] < 50) | 
                                (df_cf_unique['avg_distance_per_flight'] > 30000)]
display(extreme_avg_flight_distance)

In [0]:
# --- Exploration (Flights dispersion, Date correlations, Recency) ---

# 1) Basic uniqueness
print(f"Unique total_flights values: {df_customerflights['total_flights'].nunique():,}")

# 2) Std of flights distribution (fix possible typo 'std_flights_per_month' -> 'stdflights')
std_col = 'stdflights' if 'stdflights' in df_customerflights.columns else 'std_flights_per_month'
print(df_customerflights[std_col].describe())

# Are there any rows with std==0 but total_flights>0? (should be zero if your comment is true)
mask_inconsistent = (df_customerflights[std_col] == 0) & (df_customerflights['total_flights'] > 0)
print(f"Rows with {std_col}==0 & total_flights>0: {mask_inconsistent.sum()}")

# Distribution
plt.figure()
plt.hist(df_customerflights[std_col].fillna(0), bins=40)
plt.title(f'Distribution of {std_col}')
plt.xlabel(std_col)
plt.ylabel('Count')
plt.show()

print()

# 3) Correlations between enrollment_date and outcomes (convert dates -> numeric)
cf = df_cf_unique.copy()
cf['enrollment_date'] = pd.to_datetime(cf['enrollment_date'], errors='coerce')

# Numeric proxy for date: days since 1970-01-01 (keeps scale reasonable)
cf['enroll_days'] = (cf['enrollment_date'] - pd.Timestamp('1970-01-01')).dt.days

# Ensure numeric targets
for col in ['total_flights', 'lifetime_value']:
    cf[col] = pd.to_numeric(cf[col], errors='coerce')

pearson_corr_enroll_totalflights = cf['enroll_days'].corr(cf['total_flights'], method='pearson')
spearman_corr_enroll_totalflights = cf['enroll_days'].corr(cf['total_flights'], method='spearman')
print(f"Pearson corr(enrollment_date, total_flights): {pearson_corr_enroll_totalflights:.4f}")
print(f"Spearman corr(enrollment_date, total_flights): {spearman_corr_enroll_totalflights:.4f}")

pearson_corr_enroll_clv = cf['enroll_days'].corr(cf['lifetime_value'], method='pearson')
spearman_corr_enroll_clv = cf['enroll_days'].corr(cf['lifetime_value'], method='spearman')
print(f"Pearson corr(enrollment_date, lifetime_value): {pearson_corr_enroll_clv:.4f}")
print(f"Spearman corr(enrollment_date, lifetime_value): {spearman_corr_enroll_clv:.4f}")

print()

# 4) Recency profile (spike at 0 often = seasonality / data freshness)
rec_desc = df_cf_unique['recency'].describe()
n_zero = (df_cf_unique['recency'] == 0).sum()
print(rec_desc)
print(f"Customers with recency == 0: {n_zero:,} "
      f"({n_zero / len(df_cf_unique):.1%} of customers)")


#### Plotting the distributions (hists and boxplots)

In [0]:
# numeric features, excluding unwanted columns: loyalti_id is not necessary to plot and longitude, latitude have the same information as city and are not useful to plot as numerical values => They are useful for later geographical analysis 
exclude_cols = ["latitude", "longitude", "loyalty_id"]
metric_features = [col for col in df_cf_unique.select_dtypes(include=[np.number]).columns 
                   if col not in exclude_cols]

# TODO remove only histogram olots because we see them below next to the boxplots

def plot_histograms(df, features, bins, n_cols=3):
    """Plot clean histograms for numeric features with given number of bins."""
    sns.set_theme(style="white")  # white background
    
    n_rows = int(np.ceil(len(features) / n_cols))
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 3*n_rows))
    axes = axes.flatten()
    
    for ax, feat in zip(axes, features):
        vals = df[feat].replace([np.inf, -np.inf], np.nan).dropna()
        if len(vals) > 0:
            ax.hist(vals, bins=bins, color="steelblue", edgecolor="black")
            ax.set_title(feat, fontsize=11, pad=8)
            ax.set_xlabel("")
            ax.set_ylabel("Frequency")
        else:
            ax.set_visible(False)
    
    # hide unused subplots
    for ax in axes[len(features):]:
        ax.set_visible(False)
    
    plt.suptitle(f"Histograms of Numeric Variables ({bins} bins)", fontsize=14, y=1.02)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# two clean versions for scientific report # TODO check bins rule of thumb from data mining lab
plot_histograms(df_cf_unique, metric_features, bins=30)
plot_histograms(df_cf_unique, metric_features, bins=30)

In [0]:
# numeric features, excluding unwanted columns
exclude_cols = ["latitude", "longitude", "loyalty_id"]
metric_features = [col for col in df_cf_unique.select_dtypes(include=[np.number]).columns 
                   if col not in exclude_cols]

def plot_hist_and_box(df, features, bins=20):
    """Plot histogram + boxplot side by side for each numeric feature."""
    sns.set_theme(style="white")  # clean style
    
    for feat in features:
        vals = df[feat].replace([np.inf, -np.inf], np.nan).dropna()
        if len(vals) == 0:
            continue

        fig, axes = plt.subplots(1, 2, figsize=(10, 4), gridspec_kw={'width_ratios':[3, 1]})
        
        # Histogram
        axes[0].hist(vals, bins=bins, color="steelblue", edgecolor="black")
        axes[0].set_title(f"Histogram of {feat}", fontsize=12, pad=8)
        axes[0].set_ylabel("Frequency")
        axes[0].set_xlabel(feat)
        
        # Boxplot
        sns.boxplot(y=vals, ax=axes[1], color="steelblue")
        axes[1].set_title(f"Boxplot of {feat}", fontsize=12, pad=8)
        axes[1].set_ylabel("")
        
        # Layout
        plt.tight_layout()
        plt.show()

# Run for all numeric features
plot_hist_and_box(df_cf_unique, metric_features, bins=20)

### Bivariate Analysis

This section summarizes key distribution patterns, associations, and segment differences. We first assess rank-based associations (Spearman), then apply simple log-transformations to heavily skewed variables (columns prefixed with `log`). Afterward we revisit linear associations (Pearson). We also include categorical–categorical associations (Cramér’s V), categorical–numeric association ratio (η), pairwise numeric relationships, and compact segment comparisons.


In [0]:
# Define df for bivariate analysis
df_bivariate_analysis = df_cf_unique

numeric_cols = [
    "total_flights", "median_flights_per_month", "std_flights_per_month",
    "total_flights_w_comp", "ratio_comp_per_flight", "median_flights_w_comp", "std_flights_w_comp",
    "total_distance_km", "avg_distance_per_flight", "median_distance_km", "std_distance_km",
    "total_points_acc", "avg_points_acc_per_flight", "median_points_acc", "std_points_acc", "total_points_redeemed", "total_dollar_redeemed",
    "lifetime_value", "income", "active_months"
]

In [0]:
# Basic plot style for consistency
sns.set_theme(style='whitegrid', font_scale=1.05)
palette = sns.color_palette('deep')

def has_cols(df, cols):
    """Quick check: are all given columns in df?"""
    return all(c in df.columns for c in cols)

# Identify numeric columns.
# If a list was specified earlier, use it. Otherwise detect them.
try:
    numeric_features = numeric_cols
except NameError:
    numeric_features = [
        c for c in df_bivariate_analysis.columns
        if pd.api.types.is_numeric_dtype(df_bivariate_analysis[c])
    ]

# Categorical features I'm actually working with later.
categorical_features = [
    c for c in [
        'gender', 'marital_status', 'education',
        'loyalty_status', 'enrollment_type', 'is_enrolled'
    ]
    if c in df_bivariate_analysis.columns
]

print(f"Numeric features found: {len(numeric_features)}")
print(f"Categorical features found: {len(categorical_features)}")


#### Pairplot (All Numeric)
Pairwise relationships across numeric variables. To avoid heavy rendering on very large datasets, a small random sample is used if necessary.


In [0]:
# Pairplot only for numeric columns that are actually present
nums_exist = [c for c in numeric_features if c in df_bivariate_analysis.columns]
pp_df = df_bivariate_analysis[nums_exist].dropna()

# Avoid plotting huge data (keeps the chart readable + fast)
if len(pp_df) > 3000:
    pp_df = pp_df.sample(3000, random_state=42)

g = sns.pairplot(pp_df, diag_kind='kde', corner=True)
g.fig.suptitle('Numeric Features — Pairwise Relationships', y=1.02)
plt.show()


####Correlations

##### Spearman Correlation
Rank-based correlation across numeric variables (robust to monotonic but non-linear relationships).


In [0]:
# Spearman correlation for numeric features only
num_for_corr = [c for c in numeric_features if c in df_bivariate_analysis.columns]

if len(num_for_corr) > 1:
    corr_spear = df_bivariate_analysis[num_for_corr].dropna().corr(method='spearman')

    plt.figure(figsize=(min(0.55 * len(num_for_corr) + 5, 14),
                        min(0.55 * len(num_for_corr) + 5, 14)))

    sns.heatmap(
        corr_spear,
        annot=True,
        fmt=".2f",
        cmap="vlag",
        center=0,
        linewidths=.5,
        annot_kws={"size": 10},
        alpha=0.65,
    )

    plt.title('Spearman Correlation — Numeric Features')
    plt.tight_layout()
    plt.show()

else:
    print("Not enough numeric features for a correlation matrix.")


##### Interpretation of Spearman Correlation — Key Insights

### Strong Correlations (as expected)
- **Flight activity metrics strongly interrelated**
  - `total_flights` ↔ `median_flights_per_month` / `std_fllights_per_month` (ρ ≈ 0.72–0.93)
  - → Frequent flyers travel regularly and consistently
- **More flights → more distance → more points**
  - `total_distance_km` ↔ `total_flights` (ρ ≈ 0.80)
  - `total_points_acc` ↔ `total_distance_km` (ρ ≈ 1.00)
  - → Loyalty points earned scale with travel volume
- **Points redemption relates to engagement**
  - `total_points_redeemed` shows strong correlations with flights, distance, and points accumulated
  - → Customers who travel more also redeem significantly more points
- **Customer tenure drives value**
  - `active_months` ↔ `total_flights` (ρ ≈ 0.89)
  - → Longer relationships → more activity
- **Competing flights correlate with own flights**
  - `total_flights_w_comp` ↔ `total_flights` (ρ ≈ 0.73)
  - → High-value travelers diversify across airlines → retention challenge


### Unexpectedly Weak Correlations
- **Income shows almost no relationship with travel**
  - `income` ↔ flights/points (ρ ≈ 0.00–0.05)
  - → Might be influenced by business travelers or noisy income ranges
- **Lifetime Value not strongly tied to activity**
  - `lifetime_value` ↔ `total_flights` (ρ ≈ 0.01)
  - → Suggests other LTV components beyond pure flight revenue (or maybe not?)


> Overall, travel volume, distance, and tenure dominate loyalty value —  
> while income and redemption do **not** predict behavior as expected.


#### Transformations (Log) and Distribution Comparison
For variables that were previously transformed (columns named like `log_<var>`), we show a before/after comparison.


In [0]:
# Some of the Data is skewed to the right according to the histograms above and therefore hurting the normal distribution assumption of the pearson correlation
## we use log transformation to approximate a normal distribution

df_bivariate_analysis["log(total_points_redeemed)"] = np.log(df_bivariate_analysis["total_points_redeemed"])
df_bivariate_analysis["log(total_dollar_redeemed)"] = np.log(df_bivariate_analysis["total_dollar_redeemed"])
df_bivariate_analysis["log(lifetime_value)"] = np.log(df_bivariate_analysis["lifetime_value"])
df_bivariate_analysis["log(income)"] = np.log(df_bivariate_analysis["income"])

#drop old columns that were transformed into the new ones
df_bivariate_analysis = df_bivariate_analysis.drop(df_bivariate_analysis[["income",
                                                                      "lifetime_value",
                                                                      "total_dollar_redeemed",
                                                                      "total_points_redeemed"]] 
                                                                      , axis=1)


numeric_cols = [
    "total_flights", "median_flights_per_month", "std_flights_per_month",
    "total_flights_w_comp", "ratio_comp_per_flight", "median_flights_w_comp", "std_flights_w_comp",
    "total_distance_km", "avg_distance_per_flight", "median_distance_km", "std_distance_km",
    "total_points_acc", "avg_points_acc_per_flight", "median_points_acc", "std_points_acc",
    "log(total_points_redeemed)", "log(total_dollar_redeemed)",
    "log(lifetime_value)", "log(income)", "active_months"
]



##### Pearson Correlation (post-transform)
Linear correlation across numeric variables after accounting for skew through the available log-transformed features.


In [0]:
num_for_pearson = []
for col in numeric_cols:
    # try to find a log version formatted either style
    log_paren = f"log({col})"
    log_under = f"log_{col}"
    
    if log_paren in df_bivariate_analysis.columns:
        num_for_pearson.append(log_paren)
    elif log_under in df_bivariate_analysis.columns:
        num_for_pearson.append(log_under)
    else:
        num_for_pearson.append(col)

# ensure uniqueness while preserving order
num_for_pearson = list(dict.fromkeys(num_for_pearson))

# compute correlation
corr_pear = df_bivariate_analysis[num_for_pearson].replace([np.inf, -np.inf], np.nan).dropna().corr()

plt.figure(figsize=(0.55 * len(num_for_pearson) + 5, 0.55 * len(num_for_pearson) + 5))
sns.heatmap(
    corr_pear,
    annot=True,
    fmt=".2f",
    cmap="vlag",
    center=0,
    linewidths=.5,
    alpha=0.65,
    annot_kws={"size": 10},
)
plt.title("Pearson correlation — numeric features (log where available)")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


##### Spearman vs. Pearson — Summary

There are **no major differences** between the Spearman and Pearson correlation results.

- The **strong** relationships between travel volume, distance, tenure, and points remain **consistent**
- The **weak** relationships for `log(lifetime_value)` and `log(income)` also stay **weak**
- Log-transformations helped normalize distributions, but did **not** fundamentally change linear vs. monotonic patterns

 **Conclusion**:  
> The core relationships in our dataset are both **linear** and **monotonic**, so results from Spearman and Pearson are aligned.  
> No additional interpretation changes are necessary.


#### Boxplots: Categorical → Numeric (2×2)
Segment-wise distribution comparisons for key metrics.


In [0]:
# Fixed categorical feature list (based on your heatmap)
cat_cols = [
    "gender",
    "marital_status",
    "education",
    "loyalty_status",
    "enrollment_type",
    "is_enrolled"
]

# Relevant numeric variables you already defined/used
relevant_num_cols = [
    "total_flights", "income", "active_months", "lifetime_value", "total_points_redeemed"
]
relevant_num_cols = [c for c in relevant_num_cols if c in df_cf_unique.columns]

sns.set(style="whitegrid")
sns.set_palette("vlag")  

# One figure (grid) per categorical variable
for cat in cat_cols:
    n = len(relevant_num_cols)
    rows = math.ceil(n / 3)
    cols = min(3, n)

    fig, axes = plt.subplots(rows, cols, figsize=(16, 5 * rows))
    axes = axes.flatten()

    for ax, num in zip(axes, relevant_num_cols):
        sns.boxplot(
            data=df_cf_unique,
            x=cat,
            y=num,
            ax=ax
        )
        ax.set_title(f"{num} by {cat}")
        ax.tick_params(axis="x", rotation=25)

    for ax in axes[n:]:
        ax.set_visible(False)

    fig.suptitle(f"Numeric feature distributions by {cat}", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()


##### Boxplot Interpretation — Summary

- **Active customers fly more** → expected and clearly visible  
- **Standard enrollment leads to more flights** than promotion-based sign-ups  
- **Loyalty status relates to lifetime value** → higher tier = higher value  
- **Education clearly relates to income**, marital status slightly so  

However:
- **Income and lifetime value do not correlate** with the main travel metrics  
  (as seen in the Pearson/Spearman correlations earlier)



#### Stacked Bars (Composition, %)
Compositional view of key category interactions.


##### Why stacked percent bar charts?

They show **how one categorical variable is distributed within another**  
→ useful to spot **group differences**, **patterns**, and **imbalances** in the data.

(Visual complement to Cramér’s V)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

# Key categorical variables to include
cat_cols = [
    "gender",
    "marital_status",
    "education",
    "loyalty_status",
    "enrollment_type",
    "is_enrolled"
]

# Percent helper
def stacked_percent(df, group_col, stack_col):
    tmp = df[[group_col, stack_col]].dropna()
    if tmp.empty:
        return None
    return (
        tmp.groupby([group_col, stack_col]).size()
        .groupby(level=0).apply(lambda x: x / x.sum() * 100)
        .unstack(fill_value=0)
    )

# limited key pairings (more relevant for readability)
pairings = [
    ("education", "loyalty_status"),
    ("gender", "enrollment_type"),
    ("education", "gender"),
    ("is_enrolled", "loyalty_status"),
    ("marital_status", "education"),
    ("enrollment_type", "is_enrolled"),
    ("gender", "loyalty_status"),
    ("marital_status", "enrollment_type")
]

fig, axes = plt.subplots(4, 2, figsize=(15, 18))
axes = axes.ravel()

sns.set_style("whitegrid")
sns.set_palette("vlag")  

for ax, (g, s) in zip(axes, pairings):
    dist = stacked_percent(df_bivariate_analysis, g, s)
    if dist is not None:
        dist.plot(kind="bar", stacked=True, ax=ax, alpha=0.85)
        ax.set_title(f"{s} distribution by {g}")
        ax.set_xlabel(g)
        ax.set_ylabel("Percent")
        ax.tick_params(axis="x", rotation=25)
        ax.legend(title=s, bbox_to_anchor=(1.02, 1))
    else:
        ax.set_visible(False)

plt.tight_layout()
plt.show()


#### Cramér’s V (Categorical–Categorical Association)
Association strength between categorical features using bias-corrected Cramér’s V.


##### Cramér’s V — Measuring Association Between Categorical Variables

**Cramér’s V** is a statistical measure that quantifies how strongly two **categorical variables** are associated.

### Concept Overview
- It is based on the **Chi-square (χ²) statistic**, which compares the **observed frequencies** in a contingency table to the **expected frequencies** (if the variables were independent).  
- Cramér’s V normalizes this χ² value to produce a result **between 0 and 1**, making it easy to interpret.

\[
V = \sqrt{\frac{\chi^2}{n \times (k - 1)}}
\]

Where:
- **χ²** = Chi-square statistic from the contingency table  
- **n** = total number of observations  
- **k** = smaller of (number of rows, number of columns)

### Interpretation
| Value of V | Interpretation |
|-------------|----------------|
| 0.00 | No association |
| 0.10–0.30 | Weak association |
| 0.30–0.50 | Moderate association |
| > 0.50 | Strong association |

### Key Points
- Symmetric: same result whether you swap the variables.  
- Works only for **categorical–categorical** relationships.  
- Does **not** tell you the *direction* of association (only strength).  
- Useful for **feature selection** or **EDA** when dealing with non-numeric data.



> **Example:**  
> Measuring how strongly `loyalty_status` is associated with `marital_status`  
> → High V = membership levels differ systematically by marital status  
> → Low V = no mean:ingful difference


In [0]:
# Cramér's V for categorical features
cats = [c for c in categorical_features if c in df_bivariate_analysis.columns]

def cramers_v(x, y):
    tbl = pd.crosstab(x, y)
    if tbl.shape[0] < 2 or tbl.shape[1] < 2:
        return np.nan
    chi2 = chi2_contingency(tbl, correction=False)[0]
    n = tbl.values.sum()
    r, k = tbl.shape
    phi2 = chi2 / n
    # bias correction
    phi2_corr = max(0, phi2 - (k - 1)*(r - 1)/(n - 1))
    r_corr = r - (r - 1)**2/(n - 1)
    k_corr = k - (k - 1)**2/(n - 1)
    denom = max((k_corr - 1), (r_corr - 1))
    return np.sqrt(phi2_corr / denom) if denom > 0 else np.nan

if len(cats) > 1:
    m = pd.DataFrame(index=cats, columns=cats, dtype=float)

    for a in cats:
        for b in cats:
            if a == b:
                m.loc[a, b] = 1.0
            else:
                m.loc[a, b] = cramers_v(df_bivariate_analysis[a], df_bivariate_analysis[b])

    plt.figure(figsize=(min(0.7 * len(cats) + 5, 14),
                        min(0.7 * len(cats) + 5, 14)))

    sns.heatmap(
        m.astype(float),
        annot=True,
        fmt=".2f",
        cmap="vlag",
        vmin=0, vmax=1,
        linewidths=.5,
        annot_kws={"size": 10},
        alpha=0.65,
        cbar_kws={"shrink": 0.85}
    )

    plt.title("Cramér's V — categorical features")
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
else:
    print("Not enough categorical features for Cramér's V.")


##### Cramér’s V — Interpretation

- No strong associations among categorical variables.  
- Only a **weak link** between `marital_status` and `education` (V ≈ 0.21).  
- All others (`gender`, `loyalty_status`, `enrollment_type`, `is_enrolled`) show **near-zero** association.  

> Categorical features are largely independent — no redundancy or multicollinearity concerns.


In [0]:
pip install jinja2

#### Correlation Ratio (η) for Selected Pairs
We report η for selected categorical→numeric pairs.


##### Correlation Ratio (η) — Categorical → Numeric Association

The **Correlation Ratio (η)** measures how strongly a **categorical variable** explains the variance of a **numeric variable**.

- It captures **non-linear** relationships better than Pearson correlation.  
- η ranges from **0 to 1**:
  - **0** → No relationship (the numeric mean is similar across all categories)  
  - **1** → Perfect relationship (each category has a distinct numeric value)

\[
\eta = \sqrt{\frac{\text{between-group variance}}{\text{total variance}}}
\]


### Interpretation
- **High η** → The numeric variable changes meaningfully across categories  
  (e.g., `loyalty_status` strongly affects `totalpoints`)  
- **Low η** → The numeric variable is distributed similarly across all categories  
  (no practical relationship)

Used for:  
Assessing how much variation in a **continuous feature** is explained by a **categorical grouping** — e.g., how `education` affects `income`, or how `loyalty_status` influences `lifetime_value`.


In [0]:
# Correlation ratio (η) using log-transformed numeric features
def sanitize_numeric(s):
    return pd.to_numeric(s, errors="coerce").replace([np.inf, -np.inf], np.nan)

def correlation_ratio(categories, values):
    cat = pd.Series(categories)
    num = sanitize_numeric(values)
    
    mask = cat.notna() & num.notna()
    cat = cat[mask]
    num = num[mask]

    if cat.nunique() < 2:
        return np.nan

    means = num.groupby(cat).mean()
    counts = num.groupby(cat).size()
    mean_total = num.mean()

    ss_between = ((means - mean_total) ** 2 * counts).sum()
    ss_total = ((num - mean_total) ** 2).sum()

    return np.sqrt(ss_between / ss_total) if ss_total > 0 else np.nan


# mapping → log columns
pairs = [
    ("marital_status", "log(income)"),
    ("education", "log(income)"),
    ("loyalty_status", "avg_distance_per_flight"),  # already a ratio, no log needed
]

results = []

for cat_col, num_col in pairs:
    if cat_col in df_bivariate_analysis.columns and num_col in df_bivariate_analysis.columns:
        eta = correlation_ratio(df_bivariate_analysis[cat_col],
                                df_bivariate_analysis[num_col])
        results.append((f"{cat_col} → {num_col}", eta))
    else:
        results.append((f"{cat_col} → {num_col}", np.nan))


eta_df = pd.DataFrame(results, columns=["Pair", "Eta"])
display(eta_df.style.format({"Eta": "{:.3f}"}))


##### Correlation Ratio (η) — Interpretation Linked to Boxplots

- **education → log(income)** has a **strong relationship (η = 0.66)**  
  → Clearly visible in the boxplots: income high for bachelor education, low for college students and leveled for the other categories

- **marital_status → log(income)** shows a **very weak effect (η = 0.06)**  
  → Boxplots show slight differences (married/divorced slightly higher),  

- **loyalty_status → avgdistance** is **negligible (η = 0.02)**  
  → Matches the boxplots: loyalty tiers do **not** differ much in travel distance

 **Conclusion:**  
The correlation ratio confirms what we see visually —  
only the **education → income** link is truly meaningful,  
while the others show **little to no practical association**.


### Geospatial analysis

In [0]:
df_cf_unique.info()
geo_data = df_cf_unique[['prov_or_state', 'city', 'latitude', 'longitude', 'postal_code', 'location_code']]
geo_data.describe(include="all")

In [0]:
### Check for Inconsistencies in Geographical Data: Start from a few unique values to lots of unique values to ensure that everything is covered
# Find prov_or_state that appear in multiple location_code
location_code_prov = df_cf_unique.groupby('prov_or_state')['location_code'].nunique() # sums the amount of unique location_code values per prov_or_state
location_code_in_multiple_provs = location_code_prov[location_code_prov > 1].index.tolist() # filter prov_or_state that appear in multiple location_code
print(f"{len(location_code_in_multiple_provs)} provinces/states appear in multiple location codes.")

# Find cities that appear in multiple location_code
location_code_city = df_cf_unique.groupby('city')['location_code'].nunique() # sums the amount of unique location_code values per city
location_code_in_multiple_cities = location_code_city[location_code_city > 1].index.tolist() # filter cities that appear in multiple location_code
print(f"{len(location_code_in_multiple_cities)} cities appear in multiple location codes.")

# Find postal_code that appear in multiple location_code
location_code_postal = df_cf_unique.groupby('postal_code')['location_code'].nunique() # sums the amount of unique location_code values per postal_code
location_code_in_multiple_postals = location_code_postal[location_code_postal > 1].index.tolist() # filter postal_code that appear in multiple location_code
print(f"{len(location_code_in_multiple_postals)} postal codes appear in multiple location codes.")
print("==> Even on the postal codes level, each of the 3 location_codes is present in all entries (55 out of 55). This means that every postal code and therefore every city has rural, urban and suburban areas")
# TODO check whether this is really true and if so, the location code feature has to be dropped or transformed manually

# Find cities that appear in more than one province/state
city_prov = df_cf_unique.groupby('city')['prov_or_state'].nunique() # sums the amount of unique prov_or_state values per city
city_in_multiple_provs = city_prov[city_prov > 1].index.tolist() # filter cities that appear in more than one province/state
print(f"{len(city_in_multiple_provs)} cities appear in multiple provinces/states.")

# Find postal codes that appear in more than one city
postal_code_city = df_cf_unique.groupby('postal_code')['city'].nunique() # sums the amount of unique city values per postal code
postal_code_in_multiple_cities = postal_code_city[postal_code_city > 1].index.tolist() # filter postal codes that appear in more than one city
print(f"{len(postal_code_in_multiple_cities)} postal codes appear in multiple cities.")


# Find pairs of longitude and latitude that are assigned to more than one city
print(f"Unique latitude values: {df_cf_unique['latitude'].nunique()} || Unique longitude values: {df_cf_unique['longitude'].nunique()}")
unique_lat_lon_pairs = df_cf_unique[['latitude', 'longitude']].drop_duplicates() # determine the pairs by dropping duplicates
print(f"Number of unique (latitude, longitude) pairs: {len(unique_lat_lon_pairs)}")

lat_lon_city_counts = df_cf_unique.groupby(['latitude', 'longitude'])['city'].nunique() # sums the amount of unique city values per (latitude, longitude) pair
lat_lon_multiple_cities = lat_lon_city_counts[lat_lon_city_counts > 1] # filter pairs that appear in more than one city
print(f"{len(lat_lon_multiple_cities)} (latitude, longitude) pairs are assigned to more than one city.")
if len(lat_lon_multiple_cities) > 0:
    print(lat_lon_multiple_cities)
df_cf_unique.groupby(['latitude', 'longitude'])['city'].unique() # show the 1:1 match of coordinates to their cities


# Findings:
# - even on the postal codes level, each of the 3 location_codes is present in all entries (55 out of 55)

In [0]:
### Create a Geo Plot (Regional patterns in behavior)
# Create a geo plot using the pairs of latitude and longitude to see where customers fly the most according to their total distance travelled and total flights
geo_agg = df_cf_unique.groupby(['latitude', 'longitude', 'city'], as_index=False).agg({
    'total_distance_km': 'median',
    'total_flights': 'median'
}).rename(columns={'total_distance_km': 'totaldistance_median', 'total_flights': 'totalflights_median'}) # Aggregate total distance and total flights per (latitude, longitude) pair
display(geo_agg)

# Plot using scatter plot (size = total_distance_km, color = total_flights)
plt.figure(figsize=(10, 7))
scatter = plt.scatter(
    geo_agg['longitude'], geo_agg['latitude'],
    s=geo_agg['totaldistance_median'] / 500,  # You can't see a difference in the size but its telling the truth about the data because the values for total_distance_km are close to each other (Tufte requirement)
    c=geo_agg['totalflights_median'],
    cmap='viridis', alpha=0.7, edgecolor='k'
)
plt.colorbar(scatter, label='Total Flights')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Customer Flight Activity by Location\n(Size: Total Distance, Color: Total Flights)')
plt.grid(True)
plt.show()

print(geo_agg['totaldistance_median'] / 500)

# Findings:
# - In Sudbury there are by far the least median total flights

In [0]:
# Geo-Plot customer counts per (latitude, longitude) as dot size
counts = df_cf_unique.groupby(['latitude', 'longitude'], as_index=False).size().rename(columns={'size':'customer_count'}).reset_index(drop=True)
print(sum(counts['customer_count']) == len(df_cf_unique)) # check if counts sum up to total number of customers
geo_agg = geo_agg.merge(counts, on=['latitude', 'longitude'], how='left') # merge counts into geo_agg to have all relevant data in one dataframe
# Merge median income and customer lifetime value per location into geo_agg
income_ltv = df_cf_unique.groupby(['latitude', 'longitude'], as_index=False).agg({
    'income': 'median',
    'lifetime_value': 'median'
}).rename(columns={'income': 'income_median', 'lifetime_value': 'lifetime_value_median'})
geo_agg = geo_agg.merge(income_ltv, on=['latitude', 'longitude'], how='left')
display(geo_agg.sort_values('customer_count', ascending=False)) # Display top locations by customer count

# Compare scaled sd to display the feature with a bigger difference betweeen the customers: Min-max scale income_median and lifetime_value_median to compare spreads on the same scale
geo_agg['income_median_scaled'] = (geo_agg['income_median'] - geo_agg['income_median'].min()) / (geo_agg['income_median'].max() - geo_agg['income_median'].min())
geo_agg['lifetime_value_median_scaled'] = (geo_agg['lifetime_value_median'] - geo_agg['lifetime_value_median'].min()) / (geo_agg['lifetime_value_median'].max() - geo_agg['lifetime_value_median'].min())
print(f"Std (raw) income_median: {geo_agg['income_median'].std():.2f}, median_ltv: {geo_agg['lifetime_value_median'].std():.2f}")
print(f"Std (scaled 0-1) income_median: {geo_agg['income_median_scaled'].std():.4f}, median_ltv: {geo_agg['lifetime_value_median_scaled'].std():.4f}")
# => income_median has a higher spread than lifetime_value_median so we will use income_median for the color coding in the plot

# Scale sizes for visualization 
location_dot_sizes = (geo_agg['customer_count'])/2 # (np.sqrt(geo_agg['customer_count'])) * 40 # sqrt helps reduce skew from large counts

plt.figure(figsize=(10, 7))
sc = plt.scatter(
    geo_agg['longitude'], geo_agg['latitude'],
    s=location_dot_sizes,
    c=geo_agg['income_median'],
    cmap='Blues',
    vmin=0,
    vmax=geo_agg['income_median'].max(),
    alpha=0.7,
    edgecolor='k',
    linewidth=0.4
)
plt.colorbar(label='Median Income')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Number of Customers by Location (dot size ~ customer_count; color ~ income_median)')
plt.grid(True)
plt.show()

# Findings:
# - by far the most customers in Toronto, Vancouver and Montreal (which are also the 3 most-populated cities in Canada but Montreal has more inhabitants than Vancouver)
# - income_median and lifetime_value_median both do not vary much between locations # TODO find a feature that varies more for more impactful visualization

In [0]:
# Choropleth of Canada by province (customer counts)
import plotly.express as px
import requests
import json

# Aggregate by province
prov_agg = df_cf_unique.groupby('prov_or_state', as_index=False).agg(
    customer_count=('loyalty_id', 'nunique'),
    income_median=('income', 'median'),
    lifetime_value_median=('lifetime_value', 'median')
)

# Map province names to abbreviations
province_abbr = {
    "Alberta": "AB",
    "British Columbia": "BC",
    "Manitoba": "MB",
    "New Brunswick": "NB",
    "Newfoundland": "NL",
    "Nova Scotia": "NS",
    "Ontario": "ON",
    "Prince Edward Island": "PE",
    "Quebec": "QC",
    "Saskatchewan": "SK",
    "Yukon": "YT"
}
prov_agg["province_abbr"] = prov_agg["prov_or_state"].map(province_abbr)
prov_agg = prov_agg.sort_values(by="customer_count", ascending=False)
display(prov_agg)
# Downloaded map from https://gist.github.com/M1r1k/d5731bf39e1dfda5b53b4e4c560d968d/
canada_geo = json.load(open("data/canada_provinces.geo.json"))

# Create choropleth (featureidkey matches "properties.name" in this geojson)
fig = px.choropleth(
    prov_agg,
    geojson=canada_geo,
    locations="prov_or_state",
    featureidkey="properties.name",
    color="customer_count",
    color_continuous_scale="Blues",
    hover_data=["customer_count", "income_median", "lifetime_value_median", "province_abbr"],
    labels={"customer_count": "Unique customers", "province_abbr": "Abbreviation"},
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title_text="Customer Counts by Province (Canada)",
    margin={"r":0,"t":40,"l":0,"b":0},
    coloraxis_colorbar={"title":"Customers"}
)
fig.show()

# Findings:
# - By far the most customers are in Ontario, British Columbia and Quebec
# - 2 provinces are missing in the dataset: Northwest Territories (NT) and Nunavut (NU)


In [0]:
# Alternative GeoJSON
with open("data/georef-canada-province.geojson", "r") as f:
    canada_province_geo = json.load(f)

# Inspect the first feature to see available properties of the geojson
print("Available properties in GeoJSON:")
print(canada_province_geo['features'][0]['properties'].keys())
prov_names = [feat["properties"].get("prov_name_en") for feat in canada_province_geo["features"]]
print(sorted({name for name in prov_names if name}))
print(sorted(prov_agg["prov_or_state"].unique()))

# Create the choropleth an adjust featureidkey based on what is printed above
fig2 = px.choropleth(
    prov_agg,
    geojson=canada_province_geo,
    locations="prov_or_state",
    featureidkey="properties.prov_name_en",
    color="customer_count",
    color_continuous_scale="Greens",
    hover_data=["customer_count", "income_median", "lifetime_value_median", "prov_or_state"],
    labels={"customer_count": "Unique customers", "prov_or_state": "Province"},
)

fig2.update_geos(fitbounds="locations", visible=True)
fig2.update_traces(marker_line_color="black", marker_line_width=1)

fig2.update_layout(
    title_text="Customer Counts by Province (Canada, georef-canada-province.geojson)",
    margin={"r":0,"t":40,"l":0,"b":0},
    coloraxis_colorbar={"title":"Customers"},
    height=600,
    width=1400
)
fig2.show()

# Time Series Analysis

### Why Time Series Analysis?

Time series help us understand how **customer behavior changes over time**.  
By tracking metrics like flights, distance, or points per month, we can:

- Spot **seasonal travel patterns** (holidays, summer)
- Detect **behavior shifts** due to external factors (e.g., COVID-19)
- Monitor **loyalty engagement trends** (are customers flying more over time?)
- Compare how different **customer segments evolve** (e.g., income groups, education)


In [0]:
display(df_customerflights)
#print(df_customerflights.columns)

In [0]:
# Enrollment Trend Analysis: When Do Customers Join the Loyalty Program?
import plotly.express as px

# Work on a copy to avoid modifying original data
df_enroll = df_customerflights.copy()

# Ensure enrollment_date is properly recognized as datetime
df_enroll['enrollment_date'] = pd.to_datetime(df_enroll['enrollment_date'])

# Convert daily dates to monthly period for a cleaner time-based trend analysis
df_enroll['enrollment_month'] = (
    df_enroll['enrollment_date'].dt.to_period('M').dt.to_timestamp()
)

# Count number of new unique enrollments per month
df_month = (
    df_enroll.groupby('enrollment_month', as_index=False)
    .agg(num_customers=('loyalty_id', 'nunique'))
)

# Visualize enrollment volume per month
fig = px.bar(
    df_month,
    x="enrollment_month",
    y="num_customers",
    title="New Loyalty Program Enrollments per Month",
    labels={"enrollment_month": "Month", "num_customers": "New Customers"},
)

# Improve readability & interactivity
fig.update_layout(
    template="plotly_white",
    title_x=0.5,
    hovermode="x unified"
)

fig.show()


In [0]:
# Monthly Trends: Core Flight and Points Metrics
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Work on a safe copy
df_ts = df_customerflights.copy()

# Metrics to evaluate over time
metrics = [
    'num_flights',
    'num_flights_with_companions',
    'distance_km',
    'points_accumulated',
    'points_redeemed',
    'dollar_cost_points_redeemed'
]

# Aggregate customer metrics per month (mean values)
df_monthly = (
    df_ts.groupby('year_month_date', as_index=False)[metrics]
         .mean()
)

# Create subplot grid: one row per metric
fig = make_subplots(
    rows=len(metrics),
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=[f"Average {m} per Month" for m in metrics]
)

# Plot each metric as its own time series
for i, metric in enumerate(metrics, start=1):
    fig.add_trace(
        go.Scatter(
            x=df_monthly['year_month_date'],
            y=df_monthly[metric],
            mode="lines+markers",
            name=metric,
        ),
        row=i, col=1
    )

fig.update_layout(
    height=280 * len(metrics),
    title="Customer Activity Over Time (Monthly Averages)",
    title_x=0.5,
    template="plotly_white",
    hovermode="x unified",
    showlegend=False
)

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Average Value")
fig.show()


In [0]:
# Segmented Time Series: Distance by Customer Demographics
import plotly.express as px

df_seg = df_customerflights.copy()

# Income quantiles to create comparable segments
df_seg['income_quantile'] = pd.qcut(
    df_seg['income'], 
    q=4, 
    labels=[#'Q1 (low)', 
            'Q2', 
            'Q3', 
            'Q4 (high)'],
    duplicates='drop'
)

# Categorical segments to compare
segments = ['gender', 'education', 'location_code', 'income_quantile', 'marital_status']

for feature in segments:

    # Compute monthly avg distance by segment group
    df_grouped = (
        df_seg.groupby(['year_month_date', feature], as_index=False)
              .agg(avg_distance_km=('distance_km', 'mean'))
    )

    # Rolling smoothing to highlight trends (3 months)
    df_grouped['avg_distance_smooth'] = (
        df_grouped.groupby(feature)['avg_distance_km']
                  .transform(lambda x: x.rolling(3, center=True).mean())
    )

    fig = px.line(
        df_grouped,
        x="year_month_date",
        y="avg_distance_smooth",
        color=feature,
        title=f"Monthly Average Distance by {feature.capitalize()}",
        labels={"avg_distance_smooth": "Avg Distance (3-month mean)"}
    )

    fig.update_layout(
        template="plotly_white",
        title_x=0.5,
        hovermode="x unified",
        height=420,
        legend_title_text=feature.capitalize()
    )

    fig.show()


### Time Series Insights — Summary

Across all flight-related metrics, we observe:

**Strong seasonal patterns**
- Clear peaks in **summer**, around **Easter**, and in **December** (holiday travel)

**Overall growing trend**
- Flight activity gradually increases over the observed years

**COVID expectation**
- A dip in early 2020 would be expected due to the pandemic
- This effect is **not clearly visible** in our data

**Customer groups behave similarly**
- Gender, education, income, etc. show **almost identical** temporal travel patterns
- → No segment-specific behavioral shifts over time

**Promotion-driven surge**
- A noticeable increase in **new loyalty enrollments in 2021**
- Likely linked to **a targeted promotion campaign** that successfully attracted new members

---

**Conclusion:**  
Customer travel behavior is driven primarily by **seasonality and overall growth**.  
The **2021 enrollment spike** reflects **campaign success**, rather than structural behavioral changes in customer segments.
